In [2]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [11]:
hits = pd.read_csv('./Data/diff_coess_summary.txt', sep='\t')
hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598643 entries, 0 to 598642
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Condition  598643 non-null  object 
 1   Gene1      598643 non-null  object 
 2   Gene2      598643 non-null  object 
 3   PCC_all    598643 non-null  float64
 4   PCC_L1out  598643 non-null  float64
 5   dPCC       598643 non-null  float64
 6   Pval       598643 non-null  float64
dtypes: float64(4), object(3)
memory usage: 32.0+ MB


In [12]:
#
# PCC_all or PCC_L1out > 0.24
# and
# |dPCC| >= 0.1
# and
# Pval < 0.0021
#

In [13]:
hits['PCC_max'] = hits[['PCC_all','PCC_L1out']].max(1)
hits.head()

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval,PCC_max
0,ABCA13_LOF,ARMCX3,TAFA5,0.222,0.122,0.100,0.001,0.222
1,ABCA13_LOF,BATF3,IKZF1,0.352,0.201,0.151,0.000,0.352
2,ABCA13_LOF,BATF3,SP110,0.247,0.120,0.127,0.001,0.247
3,ABCA13_LOF,BATF3,STAT3,0.302,0.148,0.154,0.002,0.302
4,ABCA13_LOF,IKZF1,STAT3,0.254,0.116,0.138,0.000,0.254


In [14]:
filtered_hits = hits[ (hits.PCC_max>=0.248) & ( abs(hits.dPCC) >= 0.1) ]

In [29]:
filtered_hits.drop('PCC_max',axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
filtered_hits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121939 entries, 1 to 598637
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Condition  121939 non-null  object 
 1   Gene1      121939 non-null  object 
 2   Gene2      121939 non-null  object 
 3   PCC_all    121939 non-null  float64
 4   PCC_L1out  121939 non-null  float64
 5   dPCC       121939 non-null  float64
 6   Pval       121939 non-null  float64
 7   PCC_max    121939 non-null  float64
dtypes: float64(5), object(3)
memory usage: 8.4+ MB


In [30]:
filtered_hits

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
1,ABCA13_LOF,BATF3,IKZF1,0.352,0.201,0.151,0.000
3,ABCA13_LOF,BATF3,STAT3,0.302,0.148,0.154,0.002
4,ABCA13_LOF,IKZF1,STAT3,0.254,0.116,0.138,0.000
13,ABCA8_LOF,AP5B1,DCST2,0.253,0.137,0.116,0.002
17,ABCA8_LOF,DCST2,PCDHB2,0.256,0.132,0.124,0.001
...,...,...,...,...,...,...,...
598624,soft_tissue,NAV3,RASSF9,0.292,0.153,0.139,0.000
598628,soft_tissue,NR2C1,PLCXD3,0.260,0.080,0.180,0.000
598630,soft_tissue,PAX3,SOX10,0.230,0.343,-0.113,0.000
598633,soft_tissue,PAX3,ZBTB18,0.289,-0.052,0.341,0.000


In [45]:
filtered_hits.to_csv('./filtered-hits-121k-PCCge248-dPCCge100.txt', sep='\t', float_format='%4.3f', index=False)

In [66]:
edges_by_feature = filtered_hits.groupby('Condition')['Pval'].count().sort_values(ascending=False)
edges_by_feature

Condition
CDH_VIM_lo     72029
Adherent       32256
TP53_GOF        9542
Female          2632
CDH_VIM_hi      2534
               ...  
WDFY3_LOF          1
WSCD2_LOF          1
RAPGEF2_LOF        1
ZBTB10_LOF         1
KMT2A_LOF          1
Name: Pval, Length: 137, dtype: int64

In [48]:
edges_by_feature.to_csv('features_edges_count.txt')

In [67]:
dropme = where( (filtered_hits.Condition=='CDH_VIM_lo') 
               | (filtered_hits.Condition=='Adherent')
               | (filtered_hits.Condition=='Female')
               | (filtered_hits.Condition=='CDH_VIM_hi') )[0]
len(dropme)

109451

In [68]:
highly_filtered_hits = filtered_hits.drop( filtered_hits.index.values[ dropme], axis=0 )

In [69]:
highly_filtered_hits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12488 entries, 1 to 598637
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Condition  12488 non-null  object 
 1   Gene1      12488 non-null  object 
 2   Gene2      12488 non-null  object 
 3   PCC_all    12488 non-null  float64
 4   PCC_L1out  12488 non-null  float64
 5   dPCC       12488 non-null  float64
 6   Pval       12488 non-null  float64
dtypes: float64(4), object(3)
memory usage: 780.5+ KB


In [70]:
highly_filtered_hits[ highly_filtered_hits.Condition=='glioma']

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
596986,glioma,AP5B1,DCST2,0.253,0.122,0.131,0.000
596988,glioma,CPD,MET,0.248,0.082,0.166,0.000
596990,glioma,GAB1,MET,0.299,0.194,0.105,0.000
596999,glioma,JUN,NFIX,0.248,0.142,0.106,0.000
597003,glioma,NTN5,SLC28A1,0.252,0.106,0.146,0.002
597007,glioma,RPRML,SGSM2,0.284,0.184,0.100,0.001
597010,glioma,TCEAL4,TMCO6,0.250,0.138,0.112,0.000


In [71]:
highly_filtered_hits[ highly_filtered_hits.Condition=='PIK3CA_GOF'].sort_values('dPCC')

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
539868,PIK3CA_GOF,OR8B3,RBM4B,0.276,0.405,-0.128,0.001
539830,PIK3CA_GOF,IRS2,PIK3CA,0.238,0.361,-0.123,0.000
539827,PIK3CA_GOF,IGF1R,PIK3CA,0.290,0.409,-0.119,0.000
539798,PIK3CA_GOF,ERVV-2,OR8B3,0.283,0.402,-0.119,0.002
539866,PIK3CA_GOF,OR8B3,PDRG1,0.185,0.301,-0.116,0.002
539831,PIK3CA_GOF,KBTBD2,PIK3CA,0.293,0.400,-0.107,0.000
539801,PIK3CA_GOF,FOXD4,OR8B3,0.174,0.277,-0.102,0.000
539778,PIK3CA_GOF,CNPY2,PIK3CA,0.179,0.279,-0.100,0.000
539805,PIK3CA_GOF,FURIN,PIK3CA,0.198,0.299,-0.100,0.000
539869,PIK3CA_GOF,PAQR3,TESK1,0.265,0.164,0.100,0.000


In [72]:
highly_filtered_hits[ highly_filtered_hits.Condition=='KRAS_GOF'].sort_values('dPCC', ascending=False)

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
538851,KRAS_GOF,KCNK13,NAT8L,0.556,0.193,0.363,0.000
538726,KRAS_GOF,DDC,NAT8L,0.425,0.155,0.271,0.000
538691,KRAS_GOF,CFAP161,ST3GAL4,0.267,0.013,0.253,0.001
538778,KRAS_GOF,FDX1,KCNK13,0.510,0.271,0.239,0.000
538796,KRAS_GOF,FZD5,WLS,0.254,0.016,0.238,0.000
...,...,...,...,...,...,...,...
538692,KRAS_GOF,CHGA,KCNK13,0.144,0.257,-0.113,0.000
538763,KRAS_GOF,EIF2S1,ZNF202,0.192,0.309,-0.117,0.000
538795,KRAS_GOF,FZD5,TMEM254,0.153,0.269,-0.117,0.000
538696,KRAS_GOF,CLDN19,PIK3R3,0.153,0.285,-0.131,0.001


In [73]:
highly_filtered_hits[ highly_filtered_hits.Condition=='NRAS_GOF'].sort_values('dPCC', ascending=False)

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
539667,NRAS_GOF,NRAS,RAF1,0.305,-0.077,0.382,0.000
539669,NRAS_GOF,NRAS,SHOC2,0.296,0.003,0.293,0.000
539636,NRAS_GOF,CSF2RA,JAK2,0.280,0.040,0.240,0.001
539634,NRAS_GOF,CSF2RA,CSF2RB,0.362,0.131,0.231,0.001
539647,NRAS_GOF,GOLGA7,NRAS,0.296,0.083,0.212,0.000
539678,NRAS_GOF,PPARGC1A,TLE2,0.325,0.195,0.130,0.002
539677,NRAS_GOF,PPARGC1A,SVOPL,0.281,0.152,0.130,0.001
539654,NRAS_GOF,H4C7,RHOD,0.262,0.134,0.128,0.001
539685,NRAS_GOF,RHOD,TEX14,0.249,0.124,0.125,0.001
539679,NRAS_GOF,PRDM1,SIK3,0.293,0.171,0.122,0.000


In [74]:
highly_filtered_hits[ highly_filtered_hits.Condition=='central_nervous_system'].sort_values('dPCC', ascending=False)

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
596386,central_nervous_system,CPD,MET,0.248,0.084,0.164,0.000
596402,central_nervous_system,NEUROD1,OTX2,0.339,0.197,0.142,0.002
596383,central_nervous_system,AP5B1,DCST2,0.253,0.122,0.132,0.001
596408,central_nervous_system,TCEAL4,TMCO6,0.250,0.133,0.117,0.000
596398,central_nervous_system,JUN,NFIX,0.248,0.143,0.105,0.000
596389,central_nervous_system,GAB1,MET,0.299,0.195,0.104,0.000


In [75]:
highly_filtered_hits[ (highly_filtered_hits.Gene1=='CPD') & (highly_filtered_hits.Gene2=='MET') ].sort_values('dPCC', ascending=False)


,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
596988,glioma,CPD,MET,0.248,0.082,0.166,0.0
596386,central_nervous_system,CPD,MET,0.248,0.084,0.164,0.0
596967,glioblastoma,CPD,MET,0.248,0.134,0.114,0.0


In [76]:
highly_filtered_hits.nlargest(20, 'dPCC')

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
597808,peripheral_nervous_system,HAND2,MYCN,0.391,-0.130,0.522,0.0
597879,peripheral_nervous_system,ISL1,MYCN,0.463,-0.052,0.515,0.0
595072,blood,ABL1,BCR,0.602,0.091,0.512,0.0
524223,CML,ABL1,BCR,0.602,0.099,0.504,0.0
597290,melanoma,BRAF,SOX10,0.591,0.088,0.502,0.0
164448,BRAF_GOF,BRAF,DUSP4,0.477,-0.003,0.480,0.0
598230,plasma_cell,HERPUD1,PIM2,0.397,-0.057,0.454,0.0
598451,plasma_cell,POU2AF1,PRDM1,0.455,0.018,0.437,0.0
597328,melanoma,DUSP4,MITF,0.396,-0.038,0.434,0.0
597079,lymphocyte,BCL6,CYB561A3,0.286,-0.145,0.431,0.0


In [77]:
highly_filtered_hits[ (highly_filtered_hits.Gene1=='MITF') & (highly_filtered_hits.Gene2=='SOX10') ].sort_values('dPCC', ascending=False)


,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
597401,melanoma,MITF,SOX10,0.578,0.182,0.396,0.0
164565,BRAF_GOF,MITF,SOX10,0.578,0.389,0.189,0.0


In [78]:
highly_filtered_hits.nsmallest(20, 'dPCC')

,Condition,Gene1,Gene2,PCC_all,PCC_L1out,dPCC,Pval
164471,BRAF_GOF,BRAF,SHOC2,0.010,0.408,-0.399,0.000
164470,BRAF_GOF,BRAF,RAF1,0.124,0.487,-0.363,0.000
164545,BRAF_GOF,MAPK1,SHOC2,0.205,0.475,-0.270,0.000
543649,TP53_GOF,ADAM18,FGFBP3,0.091,0.315,-0.224,0.000
561195,TP53_GOF,CYRIA,MVP,0.047,0.265,-0.218,0.000
546252,TP53_GOF,AOC2,ECPAS,0.070,0.287,-0.217,0.000
585027,TP53_GOF,OR51G2,PNPLA7,0.057,0.269,-0.212,0.001
164543,BRAF_GOF,MAPK1,RAF1,0.263,0.467,-0.204,0.000
550826,TP53_GOF,BTBD17,TNFRSF11A,0.173,0.376,-0.203,0.001
551128,TP53_GOF,C12orf56,FBXW12,0.063,0.262,-0.198,0.000


In [79]:
len( highly_filtered_hits.Condition.unique() )

133

In [80]:
highly_filtered_hits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12488 entries, 1 to 598637
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Condition  12488 non-null  object 
 1   Gene1      12488 non-null  object 
 2   Gene2      12488 non-null  object 
 3   PCC_all    12488 non-null  float64
 4   PCC_L1out  12488 non-null  float64
 5   dPCC       12488 non-null  float64
 6   Pval       12488 non-null  float64
dtypes: float64(4), object(3)
memory usage: 780.5+ KB


In [81]:
highly_filtered_hits.to_csv('./highly-filtered-hits-12kEdges-133features-PCCge248-dPCCge100.txt', sep='\t', float_format='%4.3f', index=False)

In [83]:
highly_filtered_edges_by_feature = highly_filtered_hits.groupby('Condition')['Pval'].count().sort_values(ascending=False)
highly_filtered_edges_by_feature

Condition
TP53_GOF       9542
blood           455
Suspension      299
plasma_cell     207
TP53_LOF        118
               ... 
WSCD2_LOF         1
PPM1D_LOF         1
RP1L1_LOF         1
ZNF644_LOF        1
KRT14_LOF         1
Name: Pval, Length: 133, dtype: int64

In [84]:
highly_filtered_edges_by_feature.median()

4.0

In [85]:
highly_filtered_edges_by_feature.to_csv('highly_filtered_features_edges_count.txt', sep='\t')